In [1]:
from qiskit.transpiler import InstructionDurations
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_jakarta')

In [2]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [3]:
from src.tools.DD_insertion import construct_bv_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 7):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 7):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 7):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [ ]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [ ]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

In [ ]:
for job_id in bv_job_ids:
    print(job_id)

In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '628e25b0f559705bd3d5c6f9',
    '628e25b204d37422a865d6ea',
    '628e25b4e2745b027bd266cf',
    '628e25b6c7a7350423318793',


]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        pst.append(result_count[ideal_result]/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.3978271484375
pst result of hahn_X: 0.3980712890625
pst result of hahn_Y: 0.3966064453125
pst result of CP: 0.400634765625
pst result of CPMG: 0.3992919921875
pst result of xy4: 0.4019775390625
pst result of xy8: 0.40966796875
pst result of xy16: 0.4029541015625
pst result of udd1: 0.389892578125
pst result of udd2: 0.40478515625
pst result of kdd: 0.39013671875
---------Result of circuit BV4 -------------
pst result of no_dd: 0.4068603515625
pst result of hahn_X: 0.4150390625
pst result of hahn_Y: 0.3988037109375
pst result of CP: 0.42529296875
pst result of CPMG: 0.43212890625
pst result of xy4: 0.427734375
pst result of xy8: 0.431396484375
pst result of xy16: 0.42138671875
pst result of udd1: 0.42919921875
pst result of udd2: 0.431396484375
pst result of kdd: 0.427001953125
---------Result of circuit BV5 -------------
pst result of no_dd: 0.3890380859375
pst result of hahn_X: 0.393798828125
pst result of hahn_Y: 0.3

In [8]:
bv_results_data

{'BV3': [0.3978271484375,
  0.3980712890625,
  0.3966064453125,
  0.400634765625,
  0.3992919921875,
  0.4019775390625,
  0.40966796875,
  0.4029541015625,
  0.389892578125,
  0.40478515625,
  0.39013671875],
 'BV4': [0.4068603515625,
  0.4150390625,
  0.3988037109375,
  0.42529296875,
  0.43212890625,
  0.427734375,
  0.431396484375,
  0.42138671875,
  0.42919921875,
  0.431396484375,
  0.427001953125],
 'BV5': [0.3890380859375,
  0.393798828125,
  0.3792724609375,
  0.408203125,
  0.3931884765625,
  0.4114990234375,
  0.40869140625,
  0.3824462890625,
  0.411376953125,
  0.3941650390625,
  0.380859375],
 'BV6': [0.2554931640625,
  0.2581787109375,
  0.26513671875,
  0.3271484375,
  0.3228759765625,
  0.30810546875,
  0.30908203125,
  0.292236328125,
  0.324951171875,
  0.3104248046875,
  0.283447265625]}

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []

# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i/result_data[0] for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]

# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# xy16.insert(0, None)
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)
# kdd.insert(0, None)


# x_ticks = bv_circuit_names[:]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_jakarta_T.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

In [ ]:
for job_id in gs_job_ids:
    print(job_id)

In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '628e25f49fca8fff2c716b07',
    '628e25f6fdc452a94fb2b95c',
    '628e25fbf5597049d8d5c6fd',
    '628e2606732c39662fd68259',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6',]
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.019789448016555935
jsd result of hahn_X: 0.03153639265589239
jsd result of hahn_Y: 0.024528115150451214
jsd result of CP: 0.015507879527412312
jsd result of CPMG: 0.040105583352038204
jsd result of xy4: 0.04058687222912324
jsd result of xy8: 0.03705538741606715
jsd result of xy16: 0.020430532883121636
jsd result of udd1: 0.02418549965096496
jsd result of udd2: 0.03753261711631541
jsd result of kdd: 0.020033970269038505
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.055770784744942146
jsd result of hahn_X: 0.06271039684814907
jsd result of hahn_Y: 0.06034122259789145
jsd result of CP: 0.05308067270571335
jsd result of CPMG: 0.05543077425174977
jsd result of xy4: 0.0599375481244922
jsd result of xy8: 0.052598126815629365
jsd result of xy16: 0.061481265393749915
jsd result of udd1: 0.055422477483286006
jsd result of udd2: 0.06389058470078746
jsd result of kdd: 0.05668093502149895
---------Result of ci

In [11]:
gs_results_data

{'GS3': [0.019789448016555935,
  0.03153639265589239,
  0.024528115150451214,
  0.015507879527412312,
  0.040105583352038204,
  0.04058687222912324,
  0.03705538741606715,
  0.020430532883121636,
  0.02418549965096496,
  0.03753261711631541,
  0.020033970269038505],
 'GS4': [0.055770784744942146,
  0.06271039684814907,
  0.06034122259789145,
  0.05308067270571335,
  0.05543077425174977,
  0.0599375481244922,
  0.052598126815629365,
  0.061481265393749915,
  0.055422477483286006,
  0.06389058470078746,
  0.05668093502149895],
 'GS5': [0.05053178936247443,
  0.056020610371283405,
  0.05081058428416144,
  0.06182007610042269,
  0.04933010800768173,
  0.053259550859701944,
  0.0568048601360114,
  0.06458427569335526,
  0.044457867769521604,
  0.055231289571971334,
  0.0598192339930966],
 'GS6': [0.06578725817467058,
  0.06369919104674344,
  0.05884586977670418,
  0.07007025213768082,
  0.06328627596739915,
  0.07017731635046047,
  0.06372221544682251,
  0.0645397712763754,
  0.060945243460

In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)
# #
# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_jakarta.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

In [ ]:
for job_id in qft_job_ids:
    print(job_id)

In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '628e262d41fd146849a417b0',
    '628e2638c7a735c91e318798',
    '628e263b9fca8fa94d716b0a',
    '628e264b9fca8feff8716b0b',
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', ]
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.03868266715712318
jsd result of hahn_X: 0.03900376190820131
jsd result of hahn_Y: 0.03857972136646133
jsd result of CP: 0.03962436155687481
jsd result of CPMG: 0.07286878287140065
jsd result of xy4: 0.06488469803394097
jsd result of xy8: 0.053374794723515855
jsd result of xy16: 0.04643303008537045
jsd result of udd1: 0.03401416016859357
jsd result of udd2: 0.09455472212196042
jsd result of kdd: 0.06737630097140775
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.11286464107223226
jsd result of hahn_X: 0.10275196282883871
jsd result of hahn_Y: 0.10933642770286346
jsd result of CP: 0.11555981693402272
jsd result of CPMG: 0.10973666031273999
jsd result of xy4: 0.10548223592611485
jsd result of xy8: 0.1418225314432238
jsd result of xy16: 0.10574702145996223
jsd result of udd1: 0.11082196982487058
jsd result of udd2: 0.13666197897187143
jsd result of kdd: 0.1207093669914582
---------Result of circuit QF

In [14]:
qft_results_data

{'QFT3': [0.03868266715712318,
  0.03900376190820131,
  0.03857972136646133,
  0.03962436155687481,
  0.07286878287140065,
  0.06488469803394097,
  0.053374794723515855,
  0.04643303008537045,
  0.03401416016859357,
  0.09455472212196042,
  0.06737630097140775],
 'QFT4': [0.11286464107223226,
  0.10275196282883871,
  0.10933642770286346,
  0.11555981693402272,
  0.10973666031273999,
  0.10548223592611485,
  0.1418225314432238,
  0.10574702145996223,
  0.11082196982487058,
  0.13666197897187143,
  0.1207093669914582],
 'QFT5': [0.14009343256529766,
  0.14209952723510832,
  0.14279397974229796,
  0.14158527328836634,
  0.09956127142952736,
  0.12886206956831345,
  0.0992164326398858,
  0.06579942195124326,
  0.16934481376426425,
  0.11797581226455357,
  0.08579253708560398],
 'QFT6': [0.138916748232926,
  0.13805201314691537,
  0.13649136253368002,
  0.1026915867126718,
  0.09398188209374822,
  0.08007830089303548,
  0.07852694280036794,
  0.10243152698835943,
  0.10098362274146533,
  0.

In [15]:
# import numpy as np
# import matplotlib.pyplot as plt


# baseline = []
# circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6']
# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)
# #
# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp, linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_jakarta', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_jakarta.pdf', bbox_inches='tight', pad_inches=0)